In [9]:
import numpy as np
from kneed import KneeLocator
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from os.path import join


def compute_cr(raw, compressed, as_percentage=False):
    if as_percentage:
        return (1 - np.round(compressed / raw, 2)) * 100

    return np.round(raw / compressed, 2)


def get_cr_pweather():
    pweather_pmc = json.load(open(join('..', 'results', 'cr', 'pmc_pweather_cr.json')))
    pweather_swing = json.load(open(join('..', 'results', 'cr', 'swing_pweather_cr.json')))
    pweather_sz = json.load(open(join('..', 'results', 'cr', 'sz_pweather_cr.json')))

    list_cr = []

    for data, name in [(pweather_pmc, 'pweather_pmc'),
                       (pweather_swing, 'pweather_swing')]:

        uncompressed = data['OT-R']['segments']['gzip']
        df_ratio = pd.DataFrame()

        for k, v in data.items():
            lossy_ratio = {}
            for key, value in data[k]['segments'].items():
                lossy_ratio[key] = [compute_cr(uncompressed, value)]
            df_lossy = pd.DataFrame.from_dict(lossy_ratio)
            df_lossy['error_bound'] = np.round(float(k[4:]) * 0.01, 4) if len(k) > 4 else 0.0
            df_ratio = pd.concat([df_lossy, df_ratio])

        df_ratio['compression'] = name.split('_')[1]
        df_ratio['data'] = name.split('_')[0]
        list_cr.append(df_ratio)

    for data, name in [(pweather_sz, 'pweather_sz')]:
        uncompressed = data['OT-R']
        lossy_ratio = []
        error_bound = []
        for k, v in data.items():
            lossy_ratio.append(compute_cr(uncompressed, v))
            error_bound.append(float(k[1:]) if k.startswith('E') else 0)

        df_ratio = pd.DataFrame(zip(lossy_ratio, error_bound), columns=['gzip', 'error_bound'])
        df_ratio['compression'] = name.split('_')[1]
        df_ratio['data'] = name.split('_')[0]
        list_cr.append(df_ratio)

    concat_df = pd.concat(list_cr)
    concat_df = concat_df.sort_values('error_bound')
    concat_df.reset_index(inplace=True, drop=True)
    concat_df.to_csv(join('..', 'results', 'cr', 'pweather_cr.csv'), index=None)
    return concat_df


def join_pweather_cr():
    pweather = pd.read_csv('../results/tfe/pprocessed_weather.csv')
    cr_pw = get_cr_pweather()
    cr_pw['data'] = 'WEATHER'
    cr_pw.compression = cr_pw.compression.str.upper()
    ncr_pw = cr_pw.rename({'error_bound': 'eb', 'gzip': 'cr'}, axis=1).set_index(['data', 'compression', 'eb'])
    joined = pweather.join(ncr_pw, on=['data', 'compression', 'eb'])
    joined.to_csv('../results/tfe/pprocessed_weather.csv', index=False)


def process_weather():
    all_data_results = pd.read_csv('../results/tfe/all_models_results.csv')
    small_eb_weather = pd.read_csv('../results/tfe/smaller_eb_weather_all.csv', index_col=0)
    # small_eb_weather = small_eb_weather[small_eb_weather.eb > 0]
    small_eb_weather.model = small_eb_weather.model.str.upper()
    small_eb_weather.data = small_eb_weather.data.str.upper()
    small_eb_weather.eblc = small_eb_weather.eblc.str.upper()
    small_eb_weather.rename({'value': 'forecasting error', 'eblc': 'compression', 'metric': 'tfe_metric', 'tfe': 'TFE'},
                            axis=1, inplace=True)
    small_eb_weather = small_eb_weather[(small_eb_weather.te_metric == 'nrmse') & (small_eb_weather.tfe_metric == 'nrmse')]
    all_data_results = all_data_results[all_data_results.data == 'WEATHER']
    not_so_small_weather = all_data_results[
        (all_data_results.data == 'WEATHER') & (all_data_results.eb > 0.03) & (
                    all_data_results.te_metric == 'nrmse')].drop(['error', 'compression ratio'], axis=1)
    preprocessed_weather = pd.concat([small_eb_weather, not_so_small_weather])
    preprocessed_weather.to_csv('../results/tfe/processed_weather.csv', index=False)

def detect_elbow(data_name, model, eblc):
    model_results = pd.read_csv('../results/tfe/all_models_results.csv')
    model_results = model_results[(model_results.data == data_name) &
                                      (model_results.te_metric == 'nrmse') &
                                      (model_results.compression == eblc) &
                                      (model_results.model == model)]
    y = model_results.TFE.values # forecasting accuracy lost
    x = model_results.te.values  # decompression error

    # Using Kneedle to find the knee
    elbow = KneeLocator(x, y, curve='convex', direction='increasing', S=0.5)

    return elbow.elbow

def get_tfe(model_results):
    baseline_result  = model_results[model_results.eblc == 'baseline']
    model_results['tfe'] = (model_results.nrmse.values - baseline_result.nrmse.values)/baseline_result.nrmse.values
    baseline_result = baseline_result.append([baseline_result]*2, ignore_index=True)
    baseline_result.at[0, 'eblc'] = 'pmc'
    baseline_result.at[1, 'eblc'] = 'sz'
    baseline_result.at[2, 'eblc'] = 'swing'
    baseline_result['tfe'] = 0.
    return pd.concat([baseline_result, model_results])


def detect_elbow_aus(model, eblc):
    model_result = pd.read_csv(f'../results/tfe/per_model/{model}_results.csv')
    cr_results = pd.read_csv('../results/cr/all_cr.csv')
    cr_results = cr_results[(cr_results.data == 'aus') & (cr_results.compression == eblc)]
    model_result = model_result[model_result.data == 'aus']
    model_result = get_tfe(model_result)
    model_result = model_result[(model_result.eblc == eblc) & (model_result.tfe <= 0.5)]
    model_result.eb = model_result.eb.astype(float)
    uebs = model_result.eb.unique()
    if 'sz' not in eblc:
        uebs *= 0.01
        model_result.eb *= 0.01

    cr_results = cr_results[cr_results.error_bound.isin(uebs)]
    cr_results.rename({'error_bound': 'eb', 'compression': 'eblc', 'gzip': 'cr'}, axis=1, inplace=True)
    cr_results.eb = cr_results.eb.astype(float)
    cr_results.set_index(['eb', 'eblc', 'data'], inplace=True)
    model_result.set_index(['eb', 'eblc', 'data'], inplace=True)
    joined = model_result.join(cr_results)
    # display(joined)
    joined = joined.reset_index()
    y = joined.tfe.values # forecasting accuracy lost
    x = joined.error.values # decompression error
    cr = joined.cr.values
    eb = joined.eb.values
    elbow = KneeLocator(x, y, curve='convex', direction='increasing', S=1)
    # print(elbow.elbow)
    # print(joined)
    return elbow.elbow, y[np.argwhere(x == elbow.elbow)[0]], cr[np.argwhere(x == elbow.elbow)[0]], eb[np.argwhere(x == elbow.elbow)[0]]


def detect_elbow_pweather(data_name, model, eblc):
    if 'WEATHER' not in data_name:
        # print(data_name, model, eblc)
        model_results = pd.read_csv('../results/tfe/all_models_results_.csv')
        model_results = model_results[(model_results.data == data_name) &
                                          (model_results.te_metric == 'nrmse') &
                                          (model_results.compression == eblc) &
                                          (model_results.model == model)]
        y = model_results.TFE.values # forecasting accuracy lost
        x = model_results.te.values  # decompression error
        cr = model_results['compression ratio'].values
        eb = model_results.eb.values
    else:
        weather_results = pd.read_csv('../results/tfe/pweather_results.csv')
        weather_results = weather_results[(weather_results.te_metric == 'nrmse') &
                                          (weather_results.compression == eblc) &
                                          (weather_results.model == model)]

        y = weather_results.TFE.values # forecasting accuracy lost
        x = weather_results.te.values  # decompression error
        cr = weather_results.cr.values
        eb = weather_results.eb.values

    elbow = KneeLocator(x, y, curve='convex', direction='increasing', S=0.5)
    # print('Elbow', elbow.elbow)  # This will print the x value where the knee/elbow occurs


    # if 'WEATHER' in data_name:
    #     # Visualization
    #     plt.figure(figsize=(10,6))
    #     plt.plot(x, y, 'b-')
    #     plt.xlabel("Decompression Error")
    #     plt.ylabel("Forecasting Accuracy Lost")
    #     plt.vlines(elbow.elbow, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
    #     plt.show()

    return elbow.elbow, y[np.argwhere(x == elbow.elbow)[0]], cr[np.argwhere(x == elbow.elbow)[0]], eb[np.argwhere(x == elbow.elbow)[0]]


def detect_elbow_with_xg(data_name, model, eblc):
    if 'WEATHER' not in data_name:
        # print(data_name, model, eblc)
        model_results = pd.read_csv('../results/tfe/all_models_results_with_gboost.csv')
        model_results = model_results[(model_results.data == data_name) &
                                          (model_results.te_metric == 'nrmse') &
                                          (model_results.compression == eblc) &
                                          (model_results.model == model)]
        y = model_results.TFE.values # forecasting accuracy lost
        x = model_results.te.values  # decompression error
        cr = model_results['compression ratio'].values
        eb = model_results.eb.values
    else:
        weather_results = pd.read_csv('../results/tfe/pweather_results_with_xgboost.csv')
        weather_results = weather_results[(weather_results.te_metric == 'nrmse') &
                                          (weather_results.compression == eblc) &
                                          (weather_results.model == model)]

        y = weather_results.TFE.values # forecasting accuracy lost
        x = weather_results.te.values  # decompression error
        cr = weather_results.cr.values
        eb = weather_results.eb.values

    elbow = KneeLocator(x, y, curve='convex', direction='increasing', S=0.5)
    # print('Elbow', elbow.elbow)  # This will print the x value where the knee/elbow occurs


    # if 'WEATHER' in data_name:
    #     # Visualization
    #     plt.figure(figsize=(10,6))
    #     plt.plot(x, y, 'b-')
    #     plt.xlabel("Decompression Error")
    #     plt.ylabel("Forecasting Accuracy Lost")
    #     plt.vlines(elbow.elbow, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
    #     plt.show()

    return elbow.elbow, y[np.argwhere(x == elbow.elbow)[0]], cr[np.argwhere(x == elbow.elbow)[0]], eb[np.argwhere(x == elbow.elbow)[0]]


In [11]:
all_elbows = {'data':[], 'compression':[], 'model':[], 'te':[], 'tfe':[], 'cr':[], 'eb': []}

for dataset in ['ETTM1', 'ETTM2', 'SOLAR', 'WEATHER', 'WIND', 'AUS']:
    for model in ['DLINEAR', 'GRU', 'NBEATS', 'INFORMER', 'TRANSFORMER', 'ARIMA', 'GBOOST']:
        for eblc in ['PMC', 'SWING', 'SZ']:
            if 'SOLAR' in dataset and 'GRU' in model:
                continue
            if 'SOLAR' in dataset and 'ARIMA' in model and 'SZ' in eblc:
                continue
            if 'WIND' in dataset and 'ARIMA' in model and 'SZ' in eblc:
                continue

            print(dataset, model, eblc)
            elbow, tfe, cr, eb = detect_elbow_with_xg(dataset, model, eblc)
            all_elbows['data'].append(dataset)
            all_elbows['compression'].append(eblc)
            all_elbows['model'].append(model)
            all_elbows['te'].append(elbow)
            all_elbows['tfe'].append(tfe[0])
            all_elbows['cr'].append(cr[0])
            all_elbows['eb'].append(eb[0])

elbow_df = pd.DataFrame(all_elbows)
# nogru_df = elbow_df[~((elbow_df.model == 'GRU') & (elbow_df.data == 'SOLAR'))]
elbow_df.groupby(['compression', 'data']).median()

ETTM1 DLINEAR PMC
ETTM1 DLINEAR SWING
ETTM1 DLINEAR SZ
ETTM1 GRU PMC
ETTM1 GRU SWING
ETTM1 GRU SZ
ETTM1 NBEATS PMC
ETTM1 NBEATS SWING
ETTM1 NBEATS SZ
ETTM1 INFORMER PMC
ETTM1 INFORMER SWING
ETTM1 INFORMER SZ
ETTM1 TRANSFORMER PMC
ETTM1 TRANSFORMER SWING
ETTM1 TRANSFORMER SZ
ETTM1 ARIMA PMC
ETTM1 ARIMA SWING
ETTM1 ARIMA SZ
ETTM1 GBOOST PMC
ETTM1 GBOOST SWING
ETTM1 GBOOST SZ
ETTM2 DLINEAR PMC
ETTM2 DLINEAR SWING
ETTM2 DLINEAR SZ
ETTM2 GRU PMC
ETTM2 GRU SWING
ETTM2 GRU SZ
ETTM2 NBEATS PMC
ETTM2 NBEATS SWING
ETTM2 NBEATS SZ
ETTM2 INFORMER PMC
ETTM2 INFORMER SWING
ETTM2 INFORMER SZ
ETTM2 TRANSFORMER PMC
ETTM2 TRANSFORMER SWING
ETTM2 TRANSFORMER SZ
ETTM2 ARIMA PMC
ETTM2 ARIMA SWING
ETTM2 ARIMA SZ
ETTM2 GBOOST PMC
ETTM2 GBOOST SWING
ETTM2 GBOOST SZ
SOLAR DLINEAR PMC
SOLAR DLINEAR SWING
SOLAR DLINEAR SZ
SOLAR NBEATS PMC
SOLAR NBEATS SWING
SOLAR NBEATS SZ
SOLAR INFORMER PMC
SOLAR INFORMER SWING
SOLAR INFORMER SZ
SOLAR TRANSFORMER PMC
SOLAR TRANSFORMER SWING
SOLAR TRANSFORMER SZ
SOLAR ARIMA PMC


te       tfe       cr     eb
compression data                                       
PMC         AUS      0.080625  0.581949   36.120  0.250
            ETTM1    0.025014  0.022661   10.740  0.200
            ETTM2    0.019738  0.105171    7.720  0.100
            SOLAR    0.013162  0.005023    8.000  0.300
            WEATHER  0.028911  0.017415   25.800  0.017
            WIND     0.039386  0.051196   19.560  0.150
SWING       AUS      0.171975  1.142867  133.250  0.400
            ETTM1    0.023881  0.002870    3.510  0.150
            ETTM2    0.016406  0.057070    4.750  0.070
            SOLAR    0.021600  0.044904    3.490  0.275
            WEATHER  0.030919  0.022807   10.110  0.015
            WIND     0.032647  0.019831    6.990  0.100
SZ          AUS      0.057795  0.809420   47.040  0.150
            ETTM1    0.030747  0.058338   12.020  0.200
            ETTM2    0.015360  0.074324   11.330  0.070
            SOLAR    0.007611  0.009591    9.700  0.200
            WEATHER  0.066641  0.251838   23.460  0.030
            WIND     0.018139  0.045022    8.225  0.050

In [ ]:
model_results = pd.read_csv('../results/tfe/all_models_results_with_gboost.csv')


In [44]:
all_elbows = {'data':[], 'compression':[], 'model':[], 'te':[], 'tfe':[], 'cr':[], 'eb': []}

for dataset in ['ETTM1', 'ETTM2', 'SOLAR', 'WEATHER', 'WIND']:
    for model in ['DLINEAR', 'GRU', 'NBEATS', 'INFORMER', 'TRANSFORMER', 'ARIMA']:
        for eblc in ['PMC', 'SWING', 'SZ']:
            if 'SOLAR' in dataset and 'GRU' in model:
                continue
            if 'SOLAR' in dataset and 'ARIMA' in model and 'SZ' in eblc:
                continue
            if 'WIND' in dataset and 'ARIMA' in model and 'SZ' in eblc:
                continue

            print(dataset, model, eblc)
            elbow, tfe, cr, eb = detect_elbow_pweather(dataset, model, eblc)
            all_elbows['data'].append(dataset)
            all_elbows['compression'].append(eblc)
            all_elbows['model'].append(model)
            all_elbows['te'].append(elbow)
            all_elbows['tfe'].append(tfe[0])
            all_elbows['cr'].append(cr[0])
            all_elbows['eb'].append(eb[0])

elbow_df = pd.DataFrame(all_elbows)
# nogru_df = elbow_df[~((elbow_df.model == 'GRU') & (elbow_df.data == 'SOLAR'))]
elbow_df.groupby(['compression', 'data']).median()

ETTM1 DLINEAR PMC
ETTM1 DLINEAR SWING
ETTM1 DLINEAR SZ
ETTM1 GRU PMC
ETTM1 GRU SWING
ETTM1 GRU SZ
ETTM1 NBEATS PMC
ETTM1 NBEATS SWING
ETTM1 NBEATS SZ
ETTM1 INFORMER PMC
ETTM1 INFORMER SWING
ETTM1 INFORMER SZ
ETTM1 TRANSFORMER PMC
ETTM1 TRANSFORMER SWING
ETTM1 TRANSFORMER SZ
ETTM1 ARIMA PMC
ETTM1 ARIMA SWING
ETTM1 ARIMA SZ
ETTM2 DLINEAR PMC
ETTM2 DLINEAR SWING
ETTM2 DLINEAR SZ
ETTM2 GRU PMC
ETTM2 GRU SWING
ETTM2 GRU SZ
ETTM2 NBEATS PMC
ETTM2 NBEATS SWING
ETTM2 NBEATS SZ
ETTM2 INFORMER PMC
ETTM2 INFORMER SWING
ETTM2 INFORMER SZ
ETTM2 TRANSFORMER PMC
ETTM2 TRANSFORMER SWING
ETTM2 TRANSFORMER SZ
ETTM2 ARIMA PMC
ETTM2 ARIMA SWING
ETTM2 ARIMA SZ
SOLAR DLINEAR PMC
SOLAR DLINEAR SWING
SOLAR DLINEAR SZ
SOLAR NBEATS PMC
SOLAR NBEATS SWING
SOLAR NBEATS SZ
SOLAR INFORMER PMC
SOLAR INFORMER SWING
SOLAR INFORMER SZ
SOLAR TRANSFORMER PMC
SOLAR TRANSFORMER SWING
SOLAR TRANSFORMER SZ
SOLAR ARIMA PMC
SOLAR ARIMA SWING
WEATHER DLINEAR PMC
WEATHER DLINEAR SWING
WEATHER DLINEAR SZ
WEATHER GRU PMC
WEATHER G

te       tfe      cr      eb
compression data                                       
PMC         ETTM1    0.028135  0.031443  11.980  0.2250
            ETTM2    0.019738  0.092360   7.720  0.1000
            SOLAR    0.013162  0.005220   8.000  0.3000
            WEATHER  0.028911  0.014013  25.800  0.0170
            WIND     0.039386  0.046240  19.560  0.1500
SWING       ETTM1    0.023881  0.002230   3.510  0.1500
            ETTM2    0.016406  0.053817   4.750  0.0700
            SOLAR    0.019556  0.040389   3.320  0.2500
            WEATHER  0.035367  0.027194  12.805  0.0175
            WIND     0.032647  0.015474   6.990  0.1000
SZ          ETTM1    0.027132  0.048410  11.265  0.1750
            ETTM2    0.015360  0.074324  11.330  0.0700
            SOLAR    0.007611  0.007687   9.700  0.2000
            WEATHER  0.064398  0.241356  23.170  0.0285
            WIND     0.011378  0.011638   6.860  0.0300

In [50]:
all_elbows = {'compression':[], 'model':[], 'te':[], 'tfe':[], 'cr':[], 'eb':[]}
for model in ['nbeats', 'dlinear', 'informer', 'transformer', 'arima']:
    for eblc in ['swing', 'pmc', 'sz']:
        print(model, eblc)
        try:
            elbow, tfe, cr, eb = detect_elbow_aus(model, eblc)
        except IndexError:
            print('There is no elbow')
            continue
        all_elbows['model'].append(model)
        all_elbows['compression'].append(eblc)
        all_elbows['te'].append(elbow)
        all_elbows['tfe'].append(tfe[0])
        all_elbows['cr'].append(cr[0])
        all_elbows['eb'].append(eb[0])

elbow_df = pd.DataFrame(all_elbows)
elbow_df.groupby(['compression']).median()

nbeats swing
There is no elbow
nbeats pmc
nbeats sz
dlinear swing
dlinear pmc
dlinear sz
informer swing
informer pmc
informer sz
transformer swing
transformer pmc
transformer sz
arima swing
arima pmc
arima sz


,te,tfe,cr,eb
compression,,,,
pmc,0.026016,0.130000,10.14,0.07
swing,0.016420,0.049222,4.52,0.04
sz,0.019871,0.071503,25.10,0.05


In [48]:
elbow_df

,compression,model,te,tfe,cr,eb
0,pmc,nbeats,0.026016,0.190403,10.14,0.07
1,sz,nbeats,0.019871,0.126495,25.10,0.05
2,swing,dlinear,0.020413,0.077866,5.40,0.05
3,pmc,dlinear,0.012449,0.067509,5.29,0.03
4,sz,dlinear,0.012086,0.069586,19.33,0.03
5,swing,informer,0.012427,0.078071,3.64,0.03
6,pmc,informer,0.019418,0.186608,7.72,0.05
7,sz,informer,0.012086,0.075740,19.33,0.03
8,swing,transformer,0.003920,0.020577,1.93,0.01
9,pmc,transformer,0.026016,0.130000,10.14,0.07


In [ ]:
for eblc in ['PMC', 'SWING', 'SZ']:
    df_pivot = elbow_df[elbow_df['compression'] == eblc].pivot_table(index='data', columns='model', values='elbow', aggfunc='mean')

    # Create the heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(df_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Elbow Value'})
    plt.title(f"Heatmap for {eblc} Compression")
    plt.show()

In [455]:
process_weather()